In [274]:
import pyodbc
import pandas as pd 
import requests
import geocoder
from geopy.geocoders import Nominatim
from IPython.display import clear_output
from pprint import pprint
import time
import re

### Set up SQL Connection

In [527]:
cnxn = pyodbc.connect('DSN=SQLAmlin;UID=MSAmlin;PWD=password')

In [528]:
cur = cnxn.cursor()

In [561]:
london_df['Text'].dtype

dtype('O')

In [563]:
def nlp_col(df):
    names = []
    lens = []
    for d in df.columns:
        if df[d].dtype == 'O':
            names.append(d)
            lens.append(df[d].str.len().mean())

    return names[lens.index(max(lens))]

#nlp_col = 
nlp_col(london_df)



'Text'

### Define functions for getting road name and town from raw text

In [565]:
def addr_from_text(test_text):
    road_words = ['road','street', 'mews', 'row', 'place','drive', \
                  'avenue', 'end', 'boulevard', 'close', 'cottages', \
                  'way', 'square', 'villas', 'court', 'arcade', 'terrace', \
                  'gardens', 'retreat', 'avenue', 'grove', 'rise', 'flat', \
                  'lane', 'passage', 'dale', 'triangle', 'hill', 'walk', 'yard']
    try:
        test_text = [word for word in test_text.split()]
        for i in range(len(test_text)):
            #print(i)
            if test_text[i].replace(',', '').lower() in road_words:
                surr_words = [test_text[i-2], test_text[i-1], test_text[i], test_text[i+1], test_text[i+2], test_text[i+3]]
                
                surr_words = [w for w in surr_words if w.istitle()]
                
                # takes out words after a full stop - currently would go from full stops before the 
                for i in range(len(surr_words)):
                    if surr_words[i].find('.') != -1:
                        surr_words = surr_words[:i+1]
                        break
                
                surr_words = [re.search("[a-zA-Z]+", w).group() for w in surr_words]
                #print(surr_words)
            
                # takes the first address that fits the format to avoid returning empty strings
                if len(surr_words) > 0:
                    break
        if ' '.join(surr_words) == '':
            return 'None'
        else:
            return ' '.join(surr_words)#poss_addrs
    
    except IndexError:
        return 'None'
    except:
        return 'None'

In [564]:
def geotext_address(text):
    from geotext import GeoText #pip install geotext

    roadarray = ['road','street', 'mews', 'row', 'place','drive', \
                  'avenue', 'end', 'boulevard', 'close', 'cottages', \
                  'way', 'square', 'villas', 'court', 'arcade', 'terrace', \
                  'gardens', 'retreat', 'avenue', 'grove', 'rise', 'flat', \
                  'lane', 'passage', 'dale', 'triangle', 'hill', 'walk', 'yard']
    # array of locational keywords

    #interates through the numbered keys of text dictionary
    
    address = {} # initialises address dictionary
    for word in roadarray: # iterates through each word in keywords
        try:
            search  = r'((?:\w+(?:\s+|$)){1})' + word # search for the word before the key word iterated
            road = re.search(search, text, re.IGNORECASE) #  assigns var raod to the search query ignoring case
            address['ROAD'] = road.group() # address['ROAD'] is the word before keyword+keyword
        except:
            pass

    # If address['ROAD'] still doesn't exist, assign it as UNKNOWN   
    try:
        address['ROAD']
    except:
        address['ROAD'] = 'UNKNOWN'

    # Assigns TOWN as the geotext identified city
    try:
        address['TOWN'] = GeoText(text).cities[-1] # The last entry of cities is the most granular
    except:   
        address['TOWN'] = 'UNKNOWN' # Sets it to null value if town didnt match anything
    
    address # appends the address dictionary into the final locdic
    #return address['ROAD'] + ' ' + address['TOWN']
    #locdf = pd.DataFrame(address)#.from_dict(address, orient='index') # transforms locdic into a df
    address_string = address['ROAD'] + ' '+ address['TOWN']
    return address_string

In [560]:
geotext_address('asfas sadsa Fleet Street in London sdfds ss gre gre gra park lane London sadsa Fork Street in London ')

{'ROAD': 'park lane', 'TOWN': 'London'}

In [555]:
addr_from_text('asfas sadsa fleet street in London sdfds')

'London'

### Define function for getting address parts from OSM

In [199]:
def get_full_address(part_addr):
    '''Takes an address and returns a dictionary containing the components of the address.
    Works best with a road name and town'''
    time.sleep(1)
    # This outer try/except block ensures that a dictionary of the 
    # correct form is returned if the geocoder bit doesn't work
    # so that key errors won't stop this from being applied to a dataframe column
    try:
        addr_dict = {}
        
        g = geocoder.osm(part_addr)
        try:
            addr_dict['HouseNumber'] = g.json['raw']['address']['house_number'] 
        except KeyError:
            addr_dict['HouseNumber'] = None
        
        try:
            addr_dict['Road'] = g.json['raw']['address']['road']
        except KeyError:
            addr_dict['Road'] = None
        
        # The 'town' bit of the address is sometimes stored under different keys in the json
        try:
            addr_dict['Town'] = g.json['raw']['address']['town']
        except KeyError:
            try:
                addr_dict['Town'] = g.json['raw']['address']['suburb']
            except KeyError:
                try:
                    addr_dict['Town'] = g.json['raw']['address']['city']
                except:
                    addr_dict['Town'] = None
        
        try:
            addr_dict['Postcode'] = g.json['raw']['address']['postcode']
        except KeyError:
            addr_dict['Postcode'] = None
            
        try:
            addr_dict['Lat'] = g.json['lat']
        except KeyError:
            addr_dict['Lat'] = None
        
        try:
            addr_dict['Long'] = g.json['lng']
        except KeyError:
            addr_dict['Long'] = None
        
        return addr_dict    
    
    # If geocoder lookup doesn't work then return a dictionary of 'location not found' to differentiate between the None values 
    except:
        return {'HouseNumber': 'Location not found', 'Road': 'Location not found', 'Town': 'Location not found', 'Postcode': 'Location not found', 'Lat': 'Location not found', 'Long': 'Location not found'}

    
    

### Shropshire dataset

In [6]:
shrop_query = 'select * from initial.Shropshire'

In [161]:
shrop_df = pd.read_sql(shrop_query, cnxn)

In [14]:
shrop_df

,IncidentID,detail,location,start,station ground,stop,type
0,1,"At 13:37 on Tuesday, 27 November, 2018, SFRS F...",Lower Lane Chirbury ...,"Tuesday, 27 November, 2018 13:37",Bishops Castle,"The stop message was received at Tue, 11/27/20...",Property Fire
1,2,"At 13:58 on Monday, 26 November, 2018, SFRS Fi...",Caterpillar Lancaster Road Shrewsbury,"Monday, 26 November, 2018 13:58",Shrewsbury,"The stop message was received at Mon, 11/26/20...",Property Fire
2,3,"At 18:45 on Sunday, 25 November, 2018, SFRS Fi...","Crowmere Road, Shrewsbury","Sunday, 25 November, 2018 18:45",Shrewsbury,"The stop message was received at Sun, 11/25/20...",Property Fire
3,4,"At 21:24 on Wednesday, 21 November, 2018, SFRS...",Nantyr Llangollen,"Wednesday, 21 November, 2018 21:24",OTB,Assisting North Wales Fire Service deal with a...,Barn Fire
4,5,"At 21:55 on Monday, 26 November, 2018, SFRS Fi...",Faintree Bridgnorth,"Monday, 26 November, 2018 21:55",Bridgnorth,"The stop message was received at Mon, 11/26/20...",House Fire
5,6,"At 22:33 on Monday, 26 November, 2018, SFRS Fi...","Maesbury Road, Mile Oak, Oswestry","Monday, 26 November, 2018 22:33",Oswestry,"The stop message was received at Mon, 11/26/20...",Property Fire


In [404]:
get_full_address('Gamlingay , South Cambs')

{'HouseNumber': None,
 'Road': None,
 'Town': 'Gamlingay',
 'Postcode': None,
 'Lat': 52.15453115,
 'Long': -0.183294060619821}

In [405]:
shrop_df['AddressDict'] = shrop_df['location'].map(lambda x: get_full_address(x))

shrop_df['HouseNumber'] = shrop_df['AddressDict'].map(lambda x: x['HouseNumber'])
shrop_df['Road'] = shrop_df['AddressDict'].map(lambda x: x['Road'])
shrop_df['Town'] = shrop_df['AddressDict'].map(lambda x: x['Town'])
shrop_df['Postcode'] = shrop_df['AddressDict'].map(lambda x: x['Postcode'])
shrop_df['Lat'] = shrop_df['AddressDict'].map(lambda x: x['Lat'])
shrop_df['Long'] = shrop_df['AddressDict'].map(lambda x: x['Long'])

In [406]:
shrop_df

,IncidentID,detail,location,start,station ground,stop,type,HouseNumber,Road,Town,Postcode,Lat,Long,fulladdress,AddressDict
0,1,"At 13:37 on Tuesday, 27 November, 2018, SFRS F...",Lower Lane Chirbury ...,"Tuesday, 27 November, 2018 13:37",Bishops Castle,"The stop message was received at Tue, 11/27/20...",Property Fire,Location not found,Location not found,Location not found,Location not found,Location not found,Location not found,"{'HouseNumber': 'Location not found', 'Road': ...","{'HouseNumber': 'Location not found', 'Road': ..."
1,2,"At 13:58 on Monday, 26 November, 2018, SFRS Fi...",Caterpillar Lancaster Road Shrewsbury,"Monday, 26 November, 2018 13:58",Shrewsbury,"The stop message was received at Mon, 11/26/20...",Property Fire,Location not found,Location not found,Location not found,Location not found,Location not found,Location not found,"{'HouseNumber': 'Location not found', 'Road': ...","{'HouseNumber': 'Location not found', 'Road': ..."
2,3,"At 18:45 on Sunday, 25 November, 2018, SFRS Fi...","Crowmere Road, Shrewsbury","Sunday, 25 November, 2018 18:45",Shrewsbury,"The stop message was received at Sun, 11/25/20...",Property Fire,None,Crowmere Road,Shrewsbury,SY2 5JD,52.7105,-2.72995,"{'HouseNumber': 'Location not found', 'Road': ...","{'HouseNumber': None, 'Road': 'Crowmere Road',..."
3,4,"At 21:24 on Wednesday, 21 November, 2018, SFRS...",Nantyr Llangollen,"Wednesday, 21 November, 2018 21:24",OTB,Assisting North Wales Fire Service deal with a...,Barn Fire,Location not found,Location not found,Location not found,Location not found,Location not found,Location not found,"{'HouseNumber': 'Location not found', 'Road': ...","{'HouseNumber': 'Location not found', 'Road': ..."
4,5,"At 21:55 on Monday, 26 November, 2018, SFRS Fi...",Faintree Bridgnorth,"Monday, 26 November, 2018 21:55",Bridgnorth,"The stop message was received at Mon, 11/26/20...",House Fire,Location not found,Location not found,Location not found,Location not found,Location not found,Location not found,"{'HouseNumber': 'Location not found', 'Road': ...","{'HouseNumber': 'Location not found', 'Road': ..."
5,6,"At 22:33 on Monday, 26 November, 2018, SFRS Fi...","Maesbury Road, Mile Oak, Oswestry","Monday, 26 November, 2018 22:33",Oswestry,"The stop message was received at Mon, 11/26/20...",Property Fire,None,Maesbury Road,Oswestry,SY10 8NR,52.8422,-3.03886,"{'HouseNumber': 'Location not found', 'Road': ...","{'HouseNumber': None, 'Road': 'Maesbury Road',..."


### Cambridgeshire dataset

In [178]:
cambs_query = 'select * from initial.CambridgeshireFire'

In [217]:
cambs_df = pd.read_sql(cambs_query, cnxn)

In [218]:
# get rid of weird characters
cambs_df['title'] = cambs_df['title'].map(lambda x: x.replace('â€“', '-'))
# incident type stored with address - split on ' - ' to separate, then drop the first list item, then join to connect elements of address
cambs_df['location'] = cambs_df['title'].map(lambda x: ', '.join(x.split(' - ')[1:]))
# when locations have only 1 word, put cambridgeshire in the address to increase chance of OSM finding the right one
cambs_df['location'] = cambs_df['location'].map(lambda x: x + ', Cambridgeshire' if (x.count(', ') == 0)  else x)


In [219]:
cambs_df

,IncidentID,date,text,title,location
0,1,01/01/2016 - 7.38PM,Two crews from Cambridge were called to a hous...,House fire - Arbury,"Arbury, Cambridgeshire"
1,2,01/01/2017 - 12.39AM,A crew from Chatteris was called to a shed on ...,Shed fire - Doddington - Fenland,"Doddington, Fenland"
2,3,01/04/2016 - 5.52PM,A crew from Dogsthorpe was called to a house f...,House fire - Bluebell Estate - Peterborough,"Bluebell Estate, Peterborough"
3,4,01/05/2016 - 1.12PM,One crew from Dogsthorpe and one crew from Sta...,Shed fire - city centre - Peterborough,"city centre, Peterborough"
4,5,01/05/2016 - 5.52PM,One crew from Chatteris and two crews from Mar...,House fire - Chatteris - Fenland,"Chatteris, Fenland"
5,6,01/06/2016 - 1.10PM,A crew from Ely was called to a washing machin...,Washing machine fire - Littleport - East Cambs,"Littleport, East Cambs"
6,7,01/06/2016 - 6.56PM,A crew from Linton was called to a chimney fir...,Chimney fire - Weston Colville - South Cambs,"Weston Colville, South Cambs"
7,8,01/06/2016 - 9.16PM,Crews from Huntingdon and Ramsey were called t...,Derelict building fire - Upwood - Huntingdonshire,"Upwood, Huntingdonshire"
8,9,01/06/2017 - 12.16PM,At 12.16pm on Thursday (1) one crew from Stang...,Derelict house fire - Stanground - Peterborough,"Stanground, Peterborough"
9,10,01/07/2016 - 8.20PM,A crew from Cambridge was called to a house fi...,House fire - Fen Drayton - South Cambs,"Fen Drayton, South Cambs"


### London dataset

In [494]:
london_query = 'select * from initial.LondonFireRecent'

In [221]:
london_df = pd.read_sql(london_query, cnxn)

In [263]:
test_text = london_df.iloc[3]['Text']
test_text = [word.replace(',', '') for word in test_text.split()]
#test_text

In [486]:
addr_from_text(london_df.iloc[6]['Text'])

'Stamford Street Southwark'

In [491]:
london_df['Address'] = london_df['Text'].apply(addr_from_text)

In [489]:
london_df['Address'] = london_df['Address'].map(lambda x: 'None' if (x == '') else x)

In [515]:
data = london_df
row_count = data.shape[0]
ke_count = data[data['Address'] == '! Index Error !'].shape[0]
none_count = data[data['Address'] == 'None'].shape[0]
es_count = data[data['Address'] == ''].shape[0]
success = str(round(100*((row_count-(ke_count+none_count+es_count))/row_count), 2)) + '%'

print('Success Report \nRows:{}\nKey Errors:{}\nNones:{}\nEmpty Strings:{}\nSuccess rate: {}'.format(row_count,ke_count,none_count, es_count, success))

Success Report 
Rows:1518
Key Errors:1
Nones:113
Empty Strings:37
Success rate: 90.05%


In [450]:
print('Success Report \nRows:{}\nKey Errors:{}\nNones:{}\nEmpty Strings:{}'.format(row_count,ke_count,none_count, es_count))

Success Report 
Rows:1518
Key Errors:1
Nones:0
Empty Strings:37


In [551]:
london_df

,IncidentID,Caption,Date,Location,Text,Time,Address
0,1,100 firefighters tackled Welling pub fire,31/08/2015,Westminster,Over 97 firefighters and officers tackled a fi...,31/08/2015,Bellegrove Road Welling
1,2,12 fire engines tackle Southwark apartment blo...,07/02/2017,Southwark,Twelve fire engines and 81 firefighters and of...,07/02/2017,Great Guildford Street Southwark
2,3,12 people evacuate flat fire in Bermondsey,25/11/2015,Southwark,Twelve people escaped uninjured from a fire at...,25/11/2015,Grange Road Bermondsey
3,4,120 firefighters tackle Tottenham warehouse blaze,06/11/2015,Tower Hamlets,Twenty fire engines and aroundÂ 120 firefighte...,06/11/2015,Pretoria Road Tottenham
4,5,20 fire engines and 120 firefighters attended ...,06/06/2018,Westminster,TwentyÂ fire engines and 120 firefighters and ...,06/06/2018,None
5,6,20 firefighters tackled a house fire in Romford,29/05/2018,Havering,Four fire engines and 21 firefighters and offi...,29/05/2018,Dagenham Road Rush Green
6,7,200 people evacuate from fire at Southwark hal...,15/11/2015,Southwark,Two hundred people safely evacuated from a stu...,15/11/2015,Stamford Street Southwark
7,8,21 firefighters and officers fight fast food fire,17/02/2018,Waltham Forest,Four fire engines and 21 firefighters and offi...,17/02/2018,The Avenue Chingford
8,9,21 firefighters attended the scene,05/04/2018,Kingston upon Thames,Four fire engines and 21 firefighters and offi...,05/04/2018,Galsworthy Road Kingston
9,10,21 firefighters attended the scene,05/04/2018,Lambeth,Four fire engines and 21 firefighters and offi...,05/04/2018,Larkhall Lane Stockwell


### NI dataset

In [501]:
ni_query = 'select * from initial.NorthernIrelandFires'

In [502]:
ni_df = pd.read_sql(ni_query, cnxn)

In [504]:
ni_df['Address'] = ni_df['Details'].apply(addr_from_text)

In [505]:
ni_df

,IncidentID,"Date, Time",Details,Location,Address
0,1,None,NIFRS responded to a report of a fire at a sem...,None,Navan Street Armagh
1,2,"13/06/2013, 23:19",Firefighters from Central station treated two ...,BELFAST â€“ CHILDRENâ€™S HOME - CASUALTIES,College Park Avenue
2,3,"14 August 2018, 6.03am",A NIFRS spokesperson said â€“ â€œNIFRS attende...,"New Lodge, Belfast",New Lodge Road New Lodge Belfast
3,4,"14 August 2018, 6.59am",A Northern Ireland Fire & Rescue Service (NIFR...,"Drum Road, Cookstown",Drum Road Cookstown
4,5,"14 November 2018, 8.04pm",The thoughts and sympathies of Northern Irelan...,County Antrim,Carlane Road Toomebridge Magherafelt
5,6,"14/06/2013, 10:25","Crews from Banbridge, Lurgan and Dromore were ...",BANBRIDGE â€“ TERRACE HOUSE â€“ SMOKE ALARM â€...,Grove Hill
6,7,"15 August 2018, 22.51hours",â€œNIFRS received a call at 22.51hours on 15 A...,"Meenan Square, Bogside, Derry",Meenan Square Bogside Derry
7,8,"16 November 2018, 8.59pm",The thoughts and sympathies of Northern Irelan...,Omagh,Botera Upper Road Omagh Co
8,9,"19/06/2013, 15:03","Crews from Lurgan, Dromore and Banbridge had t...","GAMBLESTOWN, LURGAN â€“ DERELICT FARM SHED",Dromore Road Gamblestown
9,10,"19/06/2013, 19:55",A 61 year old man was treated for burns to his...,NEWTOWNARDS â€“ SEMI-DETACHED HOUSE â€“ SMOKE ...,None


In [514]:
data = ni_df
row_count = data.shape[0]
ke_count = data[data['Address'] == '! Index Error !'].shape[0]
none_count = data[data['Address'] == 'None'].shape[0]
es_count = data[data['Address'] == ''].shape[0]
success = str(round(100*((row_count-(ke_count+none_count+es_count))/row_count), 2)) + '%'

print('Success Report \nRows:{}\nKey Errors:{}\nNones:{}\nEmpty Strings:{}\nSuccess rate: {}'.format(row_count,ke_count,none_count, es_count, success))

Success Report 
Rows:1416
Key Errors:0
Nones:319
Empty Strings:20
Success rate: 76.06%


### South Yorkshire dataset

In [541]:
sy_query = 'select * from initial.SouthYorkshireFire'

In [542]:
sy_df = pd.read_sql(sy_query, cnxn)

In [543]:
sy_df['Text'] = sy_df['Text'].map(lambda x: x.replace('Elm Lane', '').replace('Ã‚Â', '').replace('Â', ''))

In [540]:
sy_df

,IncidentID,Date,Text,Title
0,1,Incident date: 1 August 2017,Crews from Cudworth and Barnsley recently atte...,Crews attend house fire in Worsborough
1,2,Incident date: 1 August 2018,A family were treated in hospital after a fire...,Electrical blaze sparks hospital check ups for...
2,3,Incident date: 1 July 2018,A large disused building is currently on fire ...,Derelict building fire in Rotherham
3,4,Incident date: 1 June 2016,Five fire engines were called out to a roof fi...,Roof fire at Swallownest Miners Club
4,5,Incident date: 1 June 2016,Four fire engines have been tackling a house b...,Fire crews tackle Leighton Road house fire
5,6,Incident date: 1 March 2017,Five fire engines have been tackling a blaze a...,Scrapyard fire in Barnsley
6,7,Incident date: 1 May 2015,A large amount of rubbish was set on fire next...,Arson attack in Dinnington
7,8,Incident date: 1 May 2015,A woman in her 70s was given oxygen by fire cr...,Woman given oxygen after cooker fire
8,9,Incident date: 1 May 2015,An extension lead connected to an iron sparked...,Extension lead sparks house fire
9,10,Incident date: 1 November 2015,A man in his 60s had to be taken to hospital a...,Man taken to hospital with smoke inhalation


In [548]:
sy_df['Address'] = sy_df['Text'].apply(addr_from_text)

In [549]:
data = sy_df
row_count = data.shape[0]
ke_count = data[data['Address'] == '! Index Error !'].shape[0]
none_count = data[data['Address'] == 'None'].shape[0]
es_count = data[data['Address'] == ''].shape[0]
success = str(round(100*((row_count-(ke_count+none_count+es_count))/row_count), 2)) + '%'

print('Success Report \nRows:{}\nKey Errors:{}\nNones:{}\nEmpty Strings:{}\nSuccess rate: {}'.format(row_count,ke_count,none_count, es_count, success))

Success Report 
Rows:163
Key Errors:0
Nones:19
Empty Strings:4
Success rate: 85.89%


In [550]:
sy_df.to_csv(r'C:\Users\Default\Desktop\FireAddressTest\SY.csv')